# Libraries

In [199]:
import json, os, time, glob

from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from tqdm.notebook import tqdm

# Parameters

In [203]:
base_url = 'https://www.tokopedia.com'
initial_url = 'https://www.tokopedia.com/find/kulkas-2-pintu'

# Buka Browser

In [140]:
#driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
#driver = webdriver.Chrome()

In [141]:
driver.get(initial_url)

In [149]:
driver.get('https://tokopedia.com/find/kulkas-2-pintu?page=2')

# Get Products' Element

In [150]:
elements = driver.find_elements(By.XPATH, '//div[contains(@data-testid, "divFindProduct")]')

In [151]:
len(elements)

15

In [152]:
elements[0].text

'Ad\nDisplay Cooler RSA Topaz 600 / Topaz600 Kulkas Showcase 2 Pintu 568 L\nRp8.199.000\n4.9\n28 terjual\nRefcon Polar Nusaindo'

# Scroll the page

In [153]:
last_height = driver.execute_script("return document.body.scrollHeight")

In [154]:
while True:
    for i in range(30):
        driver.execute_script(f"window.scrollBy(0, 200);")
        #driver.execute_script(f"window.scrollBy(0, {i*last_height/20});")

        time.sleep(0.5)

    #time.sleep(3)

    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height >= last_height:
        break
    last_height = new_height

In [161]:
elements = driver.find_elements(By.XPATH, '//div[contains(@data-ssr, "findProductSSR")]')
#elements = driver.find_elements(By.XPATH, '//div[contains(@data-testid, "divFindProduct")]')

In [162]:
len(elements)

80

In [197]:
li = []
pbar = tqdm(total=len(elements))
for element in elements:
    divs = element.find_elements(By.XPATH, './a/div/div[2]/div')
    name = element.find_element(By.XPATH, './a/div/div[2]/div[1]/span').text.strip()
    price = element.find_element(By.XPATH, './a/div/div[2]/div[2]/div').text.strip()

    shop_name = element.find_element(By.XPATH, f'./a/div/div[2]/div[{str(len(divs)-1)}]/div[2]/span[1]').text.strip()

    shop = element.find_element(By.XPATH, f'./a/div/div[2]/div[{str(len(divs)-1)}]/div[2]')
    actions = ActionChains(driver)
    actions.move_to_element(shop).perform()

    shop_city = element.find_element(By.XPATH, f'./a/div/div[2]/div[{str(len(divs)-1)}]/div[2]/span[2]').text.strip()
    url = element.find_element(By.XPATH, './/a').get_attribute('href')

    li.append({
        'name': name,
        'price': price,
        'url': url,
        'shop_name': shop_name,
        'shop_city': shop_city
    })
    pbar.update(1)

df = pd.DataFrame(li)

pbar.close()

  0%|          | 0/80 [00:00<?, ?it/s]

In [198]:
df.head(20)

,name,price,url,shop_name,shop_city
0,Display Cooler RSA Topaz 600 / Topaz600 Kulkas...,Rp8.199.000,https://ta.tokopedia.com/promo/v1/clicks/8a-xg...,Refcon Polar Nusaindo,Tangerang
1,SAMSUNG RT20 Kulkas 2 Pintu dengan Coolpack 12...,Rp2.850.000,https://ta.tokopedia.com/promo/v1/clicks/8a-xg...,Samsung Grade,Semarang
2,Aqua Kulkas 2 Pintu AQR-415IM(BK) 357 Litter,Rp4.990.000,https://ta.tokopedia.com/promo/v1/clicks/8a-xg...,JURAGAN-ELEKTRONIK,Jakarta Barat
3,LG GNB 215 SQMT Kulkas (2 Pintu),Rp3.695.000,https://ta.tokopedia.com/promo/v1/clicks/8a-xg...,BM Grosir Electronic,Bekasi
4,POLYTRON PRM490X Kulkas 2 Pintu Belleza 350 Li...,Rp4.692.000,https://ta.tokopedia.com/promo/v1/clicks/8a-xg...,Darrie Store,Depok
5,cetakan es twist kulkas Sharp 2 pintu original,Rp94.900,https://www.tokopedia.com/nepitom/cetakan-es-t...,Nepitom,Jakarta Timur
6,LG Kulkas 2 Pintu GN-B215SQMT Smart Inverter L...,Rp3.599.000,https://www.tokopedia.com/britplaza/lg-kulkas-...,Britplaza,Depok
7,Panasonic NR-BB201Q-PK Kulkas 2 Pintu + Invert...,Rp4.247.000,https://www.tokopedia.com/panasonic-official/p...,Panasonic,Bekasi
8,Sharp Kulkas 2 Pintu 152L SJ-195MN-HS / SJ 195...,Rp2.599.000,https://www.tokopedia.com/bonbon-elektronik/sh...,Bonbon_Elektronik,Jakarta Utara
9,Changhong CRF 208 Kulkas 2 Pintu 165L Black - ...,Rp2.099.000,https://www.tokopedia.com/stationelectro/chang...,station electronic,Jakarta Utara


# Create Dir

In [200]:
dir_path = 'scrape_tokopedia_selenium'

if not os.path.exists(dir_path):
    os.makedirs(dir_path)

In [226]:
files = glob.glob(os.path.join(dir_path, '*'))
for file in files:
    try:
        os.remove(file)
    except: pass

# Scrape Multiple Pages

In [227]:
start_page = 1
end_page = 1
pbar = tqdm(total=end_page-start_page+1)

for page in range(start_page, end_page+1):
    url_to = initial_url + f'?page={page}'
    driver.get(url_to)

    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while True:
        for i in range(30):
            driver.execute_script(f"window.scrollBy(0, 200);")

            time.sleep(0.1)

        #time.sleep(3)

        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height >= last_height:
            break
        last_height = new_height

    elements = driver.find_elements(By.XPATH, '//div[contains(@data-ssr, "findProductSSR")]')
    #elements = driver.find_elements(By.XPATH, '//div[contains(@data-testid, "divFindProduct")]')

    li = []
    for element in elements:
        divs = element.find_elements(By.XPATH, './a/div/div[2]/div')
        name = element.find_element(By.XPATH, './a/div/div[2]/div[1]/span').text.strip()
        price = element.find_element(By.XPATH, './a/div/div[2]/div[2]/div').text.strip()

        shop_name = element.find_element(By.XPATH, f'./a/div/div[2]/div[{str(len(divs)-1)}]/div[2]/span[1]').text.strip()

        shop = element.find_element(By.XPATH, f'./a/div/div[2]/div[{str(len(divs)-1)}]/div[2]')
        actions = ActionChains(driver)
        actions.move_to_element(shop).perform()

        shop_city = element.find_element(By.XPATH, f'./a/div/div[2]/div[{str(len(divs)-1)}]/div[2]/span[2]').text.strip()
        url = element.find_element(By.XPATH, './/a').get_attribute('href')

        li.append({
            'name': name,
            'price': price,
            'url': url,
            'shop_name': shop_name,
            'shop_city': shop_city
        })

    df = pd.DataFrame(li)
    df.to_csv(os.path.join(dir_path, f'{page}.csv'), index=False, sep='\t')

    pbar.update(1)

pbar.close()

  0%|          | 0/1 [00:00<?, ?it/s]

# Combine All CSVs

In [228]:
all_files = os.listdir(dir_path)

csv_files = [file for file in all_files if file.endswith('.csv')]

In [229]:
li = []

for file in csv_files:
    df = pd.read_csv(os.path.join(dir_path, file), sep='\t')
    li.append(df)

df = pd.concat(li, axis=0, ignore_index=True)
df.to_excel(os.path.join(dir_path, 'hasil_scrape.xlsx'), index=False)

In [223]:
df

,name,price,url,shop_name,shop_city
0,Kulkas 2 Pintu SHARP SJ 195 MD SG SR 200L Low ...,Rp2.629.000,https://ta.tokopedia.com/promo/v1/clicks/8a-xg...,Dunia Electronik 12,NaN
1,LG GNF392 Kulkas Besar 2 Pintu Big 2 Door Refr...,Rp8.499.900,https://ta.tokopedia.com/promo/v1/clicks/8a-xg...,Azzaelle olshop,Jakarta Timur
2,Kulkas 2 Pintu Aqua Sanyo D251 Tanpa Bunga Es ...,Rp2.580.000,https://ta.tokopedia.com/promo/v1/clicks/8a-xg...,Maju Jaya Electronik,Jakarta Barat
3,KULKAS 2 PINTU SHARP SJ-195MN-HS NO FROST,Rp2.625.000,https://ta.tokopedia.com/promo/v1/clicks/8a-xg...,Toko PROGRESS,Kab. Bogor
4,Samsung Kulkas 2 Pintu 203 L RT19M dengan All-...,Rp3.899.000,https://ta.tokopedia.com/promo/v1/clicks/8a-xg...,Samsung Electronics Authorized Store,Jakarta Barat
...,...,...,...,...,...
155,LG GN-F372PXAK Kulkas 2 Pintu 410L Gross / 363...,Rp7.450.000,https://ta.tokopedia.com/promo/v1/clicks/8a-xg...,Berlian Abadi Elektronik,Jakarta Barat
156,KULKAS 2 PINTU INVERTER POLYTRON BELLEZA JUMBO...,Rp3.140.000,https://ta.tokopedia.com/promo/v1/clicks/8a-xg...,Indo Plaza Medan,Medan
157,POLYTRON PRM491X KULKAS 2 PINTU BELLEZA BIG 35...,Rp4.990.000,https://ta.tokopedia.com/promo/v1/clicks/8a-xg...,Saturnus elektronik,Jakarta Barat
158,SHARP| SJ-456GI-MS Kulkas 2 pintu New J-Tech i...,Rp6.500.000,https://ta.tokopedia.com/promo/v1/clicks/8a-xg...,SHARP OFFICIAL STORE KUDUS,Kab. Kudus


# Close the browser

In [224]:
#driver.quit()